In [1]:
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(ChromeDriverManager().install())

In [2]:
##HEB

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json


options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

#driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe')


mlookup={"angelo":"San Angelo","htx":"Houston","sa":"San Antonio","atx":"Austin","cc":"Corpus Christi","colleges":"College Station","waco":"Waco","victoria":"Victoria","mcallen":"McAllen"}

def hebscrape(market):
    login=market+"price31415@gmail.com"
    pw=market+"price31415!!"
    
    #driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
    driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
    driver.get("https://www.heb.com/my-account/login?originalURL=https%3A%2F%2Fwww.heb.com%2Fmy-account%2Fdash-board")
    
    driver.find_element_by_xpath('//*[@id="email"]').send_keys(login)
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(pw)
    
    driver.find_element_by_xpath('//*[@id="loginForm"]/form/div[4]/div/button/span').click()
    WebDriverWait(driver, 600).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/header/div[1]/div[2]/div[1]/form/div/input')))
    
    
                       
    brizand=("Karbach","Shiner","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
    summary=list()
    
    for b in brizand:
        suburl="https://www.heb.com/search/?q="+b+"+beer"
        driver.get(suburl)
        time.sleep(7)
        #WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="footer"]/div[3]/footer/div/div[1]/div[1]/ul/li[1]/a')))
        
        
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        if len(soup.find_all(class_='sc-ztn198-0 iNjBgW sc-cltr8j-0 kvqfRY'))>0:
            product= soup.find_all(class_='sc-ztn198-0 iNjBgW sc-cltr8j-0 kvqfRY')
            
            
            price=[]
            pkg= []
            brand=[]
            for i in product:
                try:

                    title=str(re.findall(rf'aria-label=\"{b}.+?each\.',str(i))[0])
                    title=title[12:]
                    title=title.split(',')
                    
                    brand.append(title[0])
                    price.append(title[1][:-5].strip())
                    
                    
                    title2=str(re.findall(rf'alt=\"{b}.+?\"',str(i))[0])
                    title2=title2.split(',')
                    pkg.append(title2[1][1:])
                except:
                    continue


        else:
            price_selector = soup.find_all(class_='cat-price-number')
            product= soup.find_all(class_='responsivegriditem product-grid-large-fifth product-grid-small-6')

            if len(product)==0:
                product= soup.find_all(class_='ztn198-0 gIcCcD cltr8j-0 jfvaNx')


            price=[]
            for i in price_selector:
                price.append(str(re.findall(r'\$[0-9.]*',str(i))[0]))

            pkg= []
            brand=[]
            for i in product:
                L=len(str(re.findall(rf'aria-label=\"{b}.*[^\$]',str(i))))
                brand.append(str(re.findall(rf'aria-label=\"{b}.*[^\$]',str(i)))[14:L-6])
                L=len(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i))))
                tes=str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-8:L-2]

                if tes[0]!='t':
                    pkg.append(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-8:L-2])
                else:
                    pkg.append(str(re.findall(rf'aria-label=\"{b}.*\n*\t*\t*.......',str(i)))[L-7:L-2])

                   
        print([len(pkg),len(brand),len(price)])
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"HEB","market":mlookup[market],"pkg":pkg[i],"brand":brand[i],"price":price[i]})

    driver.close()
    return summary
         

#cities=["San Angelo","Houston","San Antonio","Austin","Corpus Christi","College Station","Waco","Victoria","Mcallen"]
cities=["angelo","htx","sa","atx","cc","colleges","waco","victoria","mcallen"]

Combined = list()



for i in cities:
    Combined.append(hebscrape(i))


filename=Current_Date+"heb.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()
print("done")



[14, 14, 14]
[26, 26, 26]
[10, 10, 10]
[6, 6, 6]
[8, 8, 8]
[21, 21, 21]
[26, 26, 26]
[14, 14, 14]
[14, 14, 14]
[30, 30, 30]
[29, 29, 29]
[1, 1, 1]
[6, 6, 6]
[19, 19, 19]
[20, 20, 20]
[23, 23, 23]
[13, 13, 13]
[18, 18, 18]
[27, 27, 27]
[28, 28, 28]
[2, 2, 2]
[6, 6, 6]
[9, 9, 9]
[17, 17, 17]
[26, 26, 26]
[14, 14, 14]
[15, 15, 15]
[30, 30, 30]
[24, 24, 24]
[3, 3, 3]
[6, 6, 6]
[11, 11, 11]
[13, 13, 13]
[22, 22, 22]
[18, 18, 18]
[20, 20, 20]
[29, 29, 29]
[34, 34, 34]
[1, 1, 1]
[7, 7, 7]
[12, 12, 12]
[23, 23, 23]
[25, 25, 25]
[16, 16, 16]
[19, 19, 19]
[27, 27, 27]
[27, 27, 27]
[4, 4, 4]
[6, 6, 6]
[13, 13, 13]
[20, 20, 20]
[24, 24, 24]
[13, 13, 13]
[18, 18, 18]
[12, 12, 12]
[28, 28, 28]
[7, 7, 7]
[5, 5, 5]
[4, 4, 4]
[21, 21, 21]
[23, 23, 23]
[10, 10, 10]
[19, 19, 19]
[22, 22, 22]
[29, 29, 29]
[0, 0, 0]
[5, 5, 5]
[5, 5, 5]
[15, 15, 15]
[26, 26, 26]
[12, 12, 12]
[12, 12, 12]
[15, 15, 15]
[15, 15, 15]
[0, 0, 0]
[3, 3, 3]
[3, 3, 3]
[14, 14, 14]
[26, 26, 26]
[10, 10, 10]
[17, 17, 17]
done


In [3]:
##Total Wine

import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime

options = webdriver.ChromeOptions()

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')




def twscrape(market):
    
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
    
    url="https://www.totalwine.com/store-finder/search?q="+market.replace(" ","%20")+"&radius=15"
    driver.get(url)
    time.sleep(6)
    
    try:
        hoverclick('/html/body/div[9]/div[2]/div/button')
        time.sleep(2)
    except:
        time.sleep(2)
    
    c=0
    
    while c<10:
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        c+=1
    time.sleep(1)
    while c<10:
        ActionChains(driver).send_keys(Keys.PAGE_UP).perform()
        c+=1
    time.sleep(1)
    
    if market !="San Antonio":

        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[1]/div[2]/section/div[2]/section[2]/ol/li[1]/div[2]/div[2]/a')
        targ.click()
        time.sleep(2)
    else:
        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[1]/div[2]/section/div[2]/section[2]/ol/li[2]/div[2]/div[2]/a')
        targ.click()
        time.sleep(2)
    
    try:
        targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
        targ.click()
    except:
        try:
            driver.find_element_by_xpath('/html/body/div[9]/div[2]/div/div[1]/button/img').click()
            targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
            targ.click()
        except:
            driver.find_element_by_xpath('/html/body/div[10]/div[2]/div/div[1]/button/img').click()
            targ=driver.find_element_by_xpath('/html/body/div[1]/div/div/main/div/div[2]/div/div[2]/div/div[2]/button')
            targ.click()
        
    summary=list()    
    

    brizand=("karbach","shiner","deep%20ellum","blue%20moon","saint%20arnold","corona","michelob%20ultra","truly","white%20claw")
    
    
    
    for b in brizand:
        suburl="https://www.totalwine.com/search/all?text="+b+"&pageSize=72&department=Beer"
        driver.get(suburl)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')


        price_selector = soup.find_all(class_='price__1JvDDp_x')
        product= soup.find_all(class_='productCard__2nWxIKmi')

        price=[]
        for i in price_selector:
            price.append(str(re.findall(r'\$[0-9.]*',str(i))[0]))


        brand=[]
        for i in product:
            brand.append(str(re.findall(r'true">[A-Za-z \-]*[^\<]',str(i))[0])[6:])

        pkg= []
        for i in product:
                L=len(str(re.findall(r'<span>.*?\<',str(i))[0]))
                pkg.append(str(re.findall(r'<span>.*?\<',str(i))[0][6:L-1])) 

        print([len(pkg),len(brand),len(price)])
        
        
        
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"Total Wine","market":market,"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
    return summary







#driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)


cities=["San Antonio","Austin","Dallas","Houston","Corpus Christi"]

Combined = list()

for i in cities:
    Combined.append(twscrape(i))
     
driver.close()

filename=Current_Date+"tw.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()


print("done")

[46, 46, 46]
[58, 58, 58]
[21, 21, 21]
[11, 11, 11]
[35, 35, 35]
[24, 24, 24]
[28, 28, 28]
[29, 29, 29]
[21, 21, 21]
[43, 43, 43]
[54, 54, 54]
[18, 18, 18]
[9, 9, 9]
[36, 36, 36]
[25, 25, 25]
[21, 21, 21]
[33, 33, 33]
[27, 27, 27]
[25, 25, 25]
[45, 45, 45]
[24, 24, 24]
[12, 12, 12]
[30, 30, 30]
[22, 22, 22]
[23, 23, 23]
[32, 32, 32]
[24, 24, 24]
[45, 45, 45]
[49, 49, 49]
[15, 15, 15]
[11, 11, 11]
[60, 60, 60]
[30, 30, 30]
[30, 30, 30]
[27, 27, 27]
[20, 20, 20]
[42, 42, 42]
[47, 47, 47]
[17, 17, 17]
[9, 9, 9]
[23, 23, 23]
[30, 30, 30]
[27, 27, 27]
[30, 30, 30]
[21, 21, 21]
done


In [4]:
##Kroger
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')



def krogerscrape(market):
    login=market+"31415@gmail.com"
    pw=market+"31415!!"
    
    #driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
    driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)

    driver.get("https://www.kroger.com/signin?redirectUrl=/")
    
    time.sleep(4)
    
    driver.find_element_by_xpath('//*[@id="SignIn-emailInput"]').send_keys(login)
    driver.find_element_by_xpath('//*[@id="SignIn-passwordInput"]').send_keys(pw)
    driver.find_element_by_xpath('//*[@id="SignIn-submitButton"]').click()
    
    
    
    time.sleep(8)
                   

    brizand=("karbach","shiner","deep%20ellum","blue%20moon","saint%20arnold","corona","michelob%20ultra","truly","white%20claw")
        
    summary=list()
    
    for b in brizand:
        suburl="https://www.kroger.com/search?query="+b+"%20beer&searchType=default_search&fulfillment=all"
        driver.get(suburl)
        #WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[2]/div[2]/div[3]/div/div/div/div[1]')))
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[2]/div[2]/div[3]/div/div/div/div[1]')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        product= soup.find_all(class_='AutoGrid-cell min-w-0')
        price_selector= soup.find_all(typeof="Price")    
        
                
        price=[]
        for i in price_selector:
            
            L=len(str(re.findall(r'value="[0-9.].+meta',str(i))[0]))
            price.append(str(re.findall(r'value="[0-9.].+meta',str(i))[0])[7:L-7])
        
        pkg= []
        brand=[]
        for i in product:
            try:
                L=len(str(re.findall(r'description".*?-',str(i))[0]))
                brand.append(str(re.findall(r'description".*?-',str(i))[0])[13:L-2])
                L=len(str(re.findall(r'description".*?-',str(i))[0]))+1
                pkg.append(str(re.findall(r'description".*?h3',str(i))[0])[L:-4])

            except:
                brand.append('NA')
                pkg.append('NA')
                continue
                
        print([len(pkg),len(brand),len(price)])
        

        
        for i in range(len(price)):
            summary.append({"cdate":Current_Date,"store":"Kroger","market":market.capitalize(),"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
    driver.close()
    return summary
           

#cities=["San Angelo","Houston","San Antonio","Austin","Corpus Christi","College Station","Waco","Victoria","Mcallen"]
cities=["houston","Dallas"]

Combined = list()


for i in cities:
    Combined.append(krogerscrape(i))
    


filename=Current_Date+"kroger.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()
print("done")

[24, 24, 24]
[21, 21, 21]
[0, 0, 0]
[8, 8, 8]
[20, 20, 20]
[18, 18, 17]
[19, 19, 19]
[10, 10, 9]
[16, 16, 16]
[4, 4, 4]
[23, 23, 23]
[10, 10, 10]
[7, 7, 7]
[3, 3, 3]
[19, 19, 18]
[18, 18, 18]
[14, 14, 14]
[21, 21, 21]
done


In [5]:
##Albertsons Randalls Tom Thumb

import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime

options = webdriver.ChromeOptions()

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')


mlookup={"Dallas":"75001","Houston":"77001"}


def randyscrape(market):
    
    def clickit(xxpath):
            tries=0
            while tries<3:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xxpath)))
                    driver.find_element_by_xpath(xxpath).click()
                    time.sleep(2)
                    break
                except:
                    tries+=1   
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
        
    def randyzip(xxpath):
        action = webdriver.common.action_chains.ActionChains(driver)
        el=driver.find_element_by_xpath(xxpath)
        action.move_to_element_with_offset(el,420,200)
        action.click()
        action.perform()
        time.sleep(1)
        
    
    driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
    
    url="https://www.albertsons.com"
    driver.get(url)
    time.sleep(4)
    
    hoverclick("/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[2]/div[2]/button[3]/span/span[3]")
    
    #Change zip code
        
    
    hoverclick("/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input")
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input').send_keys(mlookup[market])
    driver.find_element_by_xpath('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input').send_keys(Keys.ENTER)

    randyzip('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input')
    randyzip('/html/body/div[2]/div/div/div[3]/div/div/div/div/div[2]/store-fulfillment-modal-unified/div/div/div/div[2]/store-fulfillment-tabs/div/div[1]/input')

    
    summary=list()    
    
    
    brizand=("Karbach","Shiner","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
        
    for b in brizand:
        
        k=b+" beer"
        searchbar=driver.find_element_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/div/div/div[3]/div[2]/div[6]/form/div/div/input")
        for L in range(20):
            searchbar.send_keys(Keys.DELETE)
            searchbar.send_keys(Keys.BACKSPACE)
        searchbar.send_keys(k)
        searchbar.send_keys(Keys.ENTER)
        time.sleep(5)
        
        c=0
        while c<10:
            ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            c+=1
        time.sleep(1)
        ActionChains(driver).send_keys(Keys.PAGE_UP).perform()
        
        #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #time.sleep(5)
        #html_content = driver.execute_script('return document.body.innerHTML')
        
        page_source = driver.page_source
        #soup = BeautifulSoup(html_content, 'lxml')
        soup = BeautifulSoup(page_source)

        product= soup.body.find_all(class_='ab-lazy loaded')
        price_selector = soup.find_all(class_='product-price-con')
        
        
        price=[]
        brand=[]
        pkg=[]
        
        for pr,br in zip(price_selector,product):
                 
            try:
                brand.append(str(re.findall(rf'{b}.*? -',str(br))[0])[:-2])
                pkg.append(str(re.findall(r'- .*?(?:Oz|FZ)',str(br))[0][2:]))
                price.append(str(re.findall(r'Your Price.*?\$[0-9.]*',str(pr))[0][18:]))
            except:
                continue
        
        
        print([len(pkg),len(brand),len(price)])
        
        
        for i in range(len(pkg)):
            summary.append({"cdate":Current_Date,"store":"Albertsons Network","market":market,"pkg":pkg[i],"brand":brand[i],"price":price[i]})
        
        
    driver.close()       
    return summary
    






#driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)


cities=["Dallas","Houston"]

Combined = list()

for i in cities:
    Combined.append(randyscrape(i))
     


filename=Current_Date+"albert.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()


print("done")



[11, 11, 11]
[29, 29, 29]
[10, 10, 10]
[7, 7, 7]
[2, 2, 2]
[26, 26, 26]
[24, 25, 24]
[19, 19, 19]
[19, 19, 19]
[22, 22, 22]
[25, 25, 25]
[0, 0, 0]
[9, 9, 9]
[13, 13, 13]
[20, 20, 20]
[21, 21, 21]
[15, 15, 15]
[14, 14, 14]
done


In [7]:
##Walmart
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import re
import datetime
import json

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
options.add_argument('--disable-infobars')
options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 3})    
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
 

Current_Date = datetime.datetime.today().strftime ('%m%d%Y')

mlookup={"Dallas":"75001","San Angelo":"76901","Houston":"77001","San Antonio":"78232",
         "Austin":"73301","Corpus Christi":"78336","College Station":"77802","Waco":"76633",
         "Victoria":"77901","McAllen":"78501"}




def wmscrape(market):
    print(market)
    def clickit(xxpath):
            tries=0
            while tries<3:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xxpath)))
                    driver.find_element_by_xpath(xxpath).click()
                    time.sleep(2)
                    break
                except:
                    tries+=1   
    def hoverclick(xxpath):
        hover = ActionChains(driver).move_to_element(driver.find_element_by_xpath(xxpath))
        action = webdriver.common.action_chains.ActionChains(driver)

        tries=0
        while tries<3:
            try:
                hover.perform()
                break
            except:
                tries+=1
        time.sleep(2)
        
        action.click()
        action.perform()
        
    def randyzip(xxpath):
        action = webdriver.common.action_chains.ActionChains(driver)
        el=driver.find_element_by_xpath(xxpath)
        action.move_to_element_with_offset(el,420,200)
        action.click()
        action.perform()
        time.sleep(1)
        
    
    login=market.lower()+"31415@gmail.com"
    pw=market.lower()+"31415!!"
    
    url="https://www.walmart.com"
    driver.get(url)
    time.sleep(4)
    
    
    
    #Change zip code
            
    hoverclick("/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[2]/div/div[1]/div[1]/button/span/img")
    hoverclick("/html/body/div[1]/div/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[3]/button/span")
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.BACKSPACE)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.BACKSPACE)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.BACKSPACE)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.BACKSPACE)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.BACKSPACE)
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(mlookup[market])
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[1]/section/section/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div[3]/div[3]/div/div[2]/form/div[1]/input').send_keys(Keys.ENTER)
    
    time.sleep(3)
    
    summary=list()    
    

    brizand=("Shiner","Karbach","Deep Ellum","Blue Moon","Saint Arnold","Corona","Michelob Ultra","Truly","White Claw")
        
    for b in brizand:
        
        k=b+" beer"
        searchbar=driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/section/section/div[2]/div/div[3]/div[2]/div/form/input[2]")
        if b != "Shiner":

            for L in range(60):
                searchbar.send_keys(Keys.DELETE)
                
            for L in range(60):
                
                searchbar.send_keys(Keys.BACKSPACE)
        
        searchbar.send_keys(k)
        searchbar.send_keys(Keys.ENTER)
        time.sleep(3)
        
        
        tiles = driver.find_elements_by_class_name("orientation-square")
        wrongproduct=0
        for t in range(len(tiles)):
            tiles1 = driver.find_elements_by_class_name("orientation-square")
            
            if wrongproduct>1:
                break
            
            tiles1[t].click()
            time.sleep(3)
            
            
            page_source = driver.page_source
            #soup = BeautifulSoup(html_content, 'lxml')
            soup = BeautifulSoup(page_source)
                        
            product= soup.body.find_all(class_='prod-ProductTitle prod-productTitle-buyBox font-bold')
            price_selector = soup.find_all(class_='price display-inline-block arrange-fit price')

            
            for i in range(len(product)):
                try:
                    
                    
                    if len(re.findall(rf'{b}.*?\"',str(product[i])))==0:
                        wrongproduct+=1
                        
                        continue
                    
                    price=[]
                    brand=[]
                    pkg= []

                    price.append(str(re.findall(r'\$[0-9.]*',str(price_selector[i]))[0]))

                    brand.append(str(re.findall(rf'{b}.*?(?:,|[0-9])',str(product[i]))[0])[:-1])
                    L=len(re.findall(rf'{b}.*?(?:\,|[0-9])',str(product[i]))[0])-1

                    pkg.append(str(re.findall(rf'{b}.*?\"',str(product[i]))[0])[L:])

                    
                    print([len(pkg),len(brand),len(price)])
                    
                    summary.append({"cdate":Current_Date,"store":"Walmart","market":market,"pkg":pkg[0],"brand":brand[0],"price":price[0]})


                except:
                    continue

          
            
            
            driver.back()
            time.sleep(3)
            
    
            
        
    
    return summary


driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)
           

#cities=["Dallas","San Angelo","Houston","San Antonio","Austin","Corpus Christi","College Station","Waco","Victoria","McAllen"]
cities=["Austin","Corpus Christi","College Station","Waco","Dallas","San Angelo","Houston","San Antonio","Victoria","McAllen"]
#
Combined = list()

trock=0

for i in cities:
    while True:
    
        try:
            Combined.append(wmscrape(i))
            break
        except:
            print("Fail")
            driver.close()
            driver = webdriver.Chrome(executable_path=r'C:\Users\MGrogan\Documents\People\todd\chromedriver.exe', options=options)

driver.close()
    
filename=Current_Date+"wm.json"
update=open(filename,'w')
json.dump(Combined,update)
update.close()

print("done")

McAllen
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
done


In [ ]:
Combined[-1]